### Pacotes necessários

In [112]:
import pandas as pd
import sys
import scipy
import matplotlib 
import seaborn
import numba
import gmpy2
import pysubgroup as ps
import numpy as np
import math
sys.path.append('./SSDpp-numeric-master')
sys.path
from src.util.results2folder import attach_results,print2folder
from _classes import SSDC

### Leitura do banco de dados

In [96]:
df = pd.read_csv('./data/dengue_internacao.csv', sep=',')

C:\Users\alexa\AppData\Local\Temp\ipykernel_10588\3527530056.py:1: DtypeWarning: Columns (4,16,21,107,113,119,142,150) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/dengue_internacao.csv', sep=',')


In [97]:
# selecionando as colunas desejadas
df_selecionado = df[['ano', 'data_notificacao', 'sigla_uf_notificacao', 'data_primeiros_sintomas', 'idade_paciente', 
                     'sexo_paciente', 'raca_cor_paciente', 'escolaridade_paciente', 'gestante_paciente', 
                     'possui_doenca_autoimune', 'possui_diabetes', 'possui_doencas_hematologicas', 
                     'possui_hepatopatias', 'possui_doenca_renal', 'possui_hipertensao', 
                     'possui_doenca_acido_peptica', 'apresenta_febre','apresenta_cefaleia','apresenta_exantema','apresenta_dor_costas','apresenta_mialgia','apresenta_vomito','apresenta_conjutivite','apresenta_dor_retroorbital',
                     'apresenta_artralgia','apresenta_artrite','apresenta_leucopenia','apresenta_petequias',
                     'data_internacao','classificacao_final','evolucao_caso', 'data_obito', 'data_encerramento']]

df_selecionado = df_selecionado[df_selecionado['ano'] >= 2023]


### Tratamento do banco de dados

In [98]:
# transformando atributo nominal data_obito para binário houve_obito
df_selecionado['houve_obito'] = df_selecionado['data_obito'].apply(lambda x: 0 if pd.isna(x) else 1)

# remoção data_obito
df_selecionado = df_selecionado.drop(columns=['data_obito'])

In [99]:
# removendo as linhas onde não houveram datas de primeiros sintomas ou data da notificação.
df_selecionado = df_selecionado.dropna(subset=['data_primeiros_sintomas', 'data_notificacao'])

df_selecionado['data_notificacao'] = pd.to_datetime(df_selecionado['data_notificacao'], errors='coerce')
df_selecionado['data_primeiros_sintomas'] = pd.to_datetime(df_selecionado['data_primeiros_sintomas'], errors='coerce')
# criando a coluna 'tempo_internacao'
df_selecionado['intervalo_busca_atendimento'] = df_selecionado.apply(
    lambda row: (row['data_notificacao'] - row['data_primeiros_sintomas']).days if pd.notna(row['data_primeiros_sintomas']) else 0,
    axis=1
)

In [100]:
# removendo as linhas onde não houveram internações ou data de encerramento.
df_selecionado = df_selecionado.dropna(subset=['data_internacao', 'data_encerramento'])

# convertendo as colunas para datetime
df_selecionado['data_encerramento'] = pd.to_datetime(df_selecionado['data_encerramento'], errors='coerce')
df_selecionado['data_internacao'] = pd.to_datetime(df_selecionado['data_internacao'], errors='coerce')

# criando a coluna 'intervalo_sintoma_internacao' que é a data dos primeiros sintomas até a data que ela foi internada.
df_selecionado['intervalo_sintoma_internacao'] = df_selecionado.apply(
    lambda row: (row['data_internacao'] - row['data_primeiros_sintomas']).days if pd.notna(row['data_internacao']) else 0,
    axis=1
)

# criando a coluna 'intervalo_internacao'
df_selecionado['intervalo_internacao'] = df_selecionado.apply(
    lambda row: (row['data_encerramento'] - row['data_internacao']).days if pd.notna(row['data_internacao']) else 0,
    axis=1
)

df_selecionado = df_selecionado.query("intervalo_internacao >= 0 and intervalo_internacao <= 90 and" + " intervalo_sintoma_internacao >= 0 and intervalo_sintoma_internacao <= 90 and"
              + " intervalo_busca_atendimento >= 0 and intervalo_busca_atendimento <= 90")

In [101]:
# prompt: na variável, idade_paciente, retirar manter os valores depois de "-" e transformar em inteiro
df_selecionado['idade_paciente'] = df_selecionado['idade_paciente'].str.split('-').str[1]
df_selecionado['idade_paciente'] = df_selecionado['idade_paciente'].str.extract('(\d+)').astype(int)

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\alexa\AppData\Local\Temp\ipykernel_10588\53800304.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_selecionado['idade_paciente'] = df_selecionado['idade_paciente'].str.extract('(\d+)').astype(int)


In [102]:
# drop das variáveis data_notificacao, data_primeiros_sintomas, data_internacao e data_encerramento
df_selecionado = df_selecionado.drop(['data_notificacao', 'data_primeiros_sintomas', 'data_internacao', 'data_encerramento'], axis=1)

In [103]:
# recodificando as variáveis categóricas
raca_cor_map = {
    1:"Branca",
    2:"Preta",
    3:"Amarela",
    4:"Parda",
    5:"Indígena",
    9:"Ignorado"
}
df_selecionado["raca_cor_paciente"] = df_selecionado["raca_cor_paciente"].map(raca_cor_map)

escolaridade_map = {
    0: "Analfabeto",
    1: "1ª a 4ª série incompleta do EF",
    2: "4ª série completa do EF",
    3: "5ª A 8ª série incomplet do EF",
    4: "Ensino fundamental incompleto",
    5: "Ensino médio incompleto",
    6: "Ensino médio completo",
    7: "Educação superior incompleta",
    8: "Educação superior completa",
    9: "Ignorado",
    10: "Não se Aplica"
}

df_selecionado["escolaridade_paciente"] = df_selecionado["escolaridade_paciente"].map(escolaridade_map)

gestante_map = {
    1:"1ºTrimestre", 2:"2ºTrimestre", 3:"3ºTrimestre", 4:"Idade gestacional Ignorada", 5:"Não", 6:"Não se aplica",9:"Ignorado"
}

df_selecionado["gestante_paciente"] = df_selecionado["gestante_paciente"].map(gestante_map)

dicotomia_map = {
    1: "Sim",
    2: "Não"
}

col_dicotomia = ['possui_doenca_autoimune', 'possui_diabetes', 'possui_doencas_hematologicas', 
                     'possui_hepatopatias', 'possui_doenca_renal', 'possui_hipertensao', 
                     'possui_doenca_acido_peptica', 'apresenta_febre','apresenta_cefaleia','apresenta_exantema','apresenta_dor_costas','apresenta_mialgia','apresenta_vomito','apresenta_conjutivite','apresenta_dor_retroorbital',
                     'apresenta_artralgia','apresenta_artrite','apresenta_leucopenia','apresenta_petequias']

for col in col_dicotomia:
    df_selecionado[col] = df_selecionado[col].map(dicotomia_map)

classificacao_map={
    5: "Descartado",
    10: "Dengue", 
    11: "Dengue com Sinais de Alarme",
    12: "Dengue Grave", 
    13: "Chikungunya"
}

df_selecionado['classificacao_final'] = df_selecionado['classificacao_final'].map(classificacao_map).fillna(np.nan)

evolucao_map={
    1:"Cura",
    2:"Óbito pelo agravo", 
    3:"Óbito por outras causas",
    4:"Óbito em investigação", 
    9:"Ignorado",
}

df_selecionado['evolucao_caso']=df_selecionado['evolucao_caso'].map(evolucao_map).fillna(np.nan)
df_selecionado['houve_obito'] = df_selecionado['houve_obito'].map({1: 'sim', 0: 'não'})

df_selecionado

,ano,sigla_uf_notificacao,idade_paciente,sexo_paciente,raca_cor_paciente,escolaridade_paciente,gestante_paciente,possui_doenca_autoimune,possui_diabetes,possui_doencas_hematologicas,...,apresenta_artralgia,apresenta_artrite,apresenta_leucopenia,apresenta_petequias,classificacao_final,evolucao_caso,houve_obito,intervalo_busca_atendimento,intervalo_sintoma_internacao,intervalo_internacao
118790,2024,AM,55,F,Parda,5ª A 8ª série incomplet do EF,Não,Não,Não,Não,...,Sim,Não,Sim,Não,NaN,NaN,não,6,6,61
118792,2024,AM,88,M,Parda,NaN,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue Grave,Óbito pelo agravo,sim,4,3,61
118793,2024,AM,54,M,Parda,Educação superior completa,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,4,1,3
118794,2024,AM,48,F,Parda,Ensino médio completo,Não,Não,Não,Não,...,Não,Não,Não,Sim,Dengue Grave,Cura,não,4,1,22
118795,2024,AM,83,F,Parda,Ensino médio incompleto,Não,Não,Não,Não,...,Não,Não,Não,Não,Dengue com Sinais de Alarme,Cura,não,1,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257951,2023,SP,54,M,Branca,Ensino médio completo,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,1,0,9
257952,2023,SP,39,M,Branca,NaN,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,4,4,1
257953,2023,SP,13,F,Branca,Ensino médio incompleto,Não se aplica,Não,Não,Não,...,Não,Não,Sim,Não,Dengue com Sinais de Alarme,Cura,não,3,4,25
257954,2023,SP,75,M,Branca,Ignorado,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,31,30,2


In [104]:
## retirando os resultados NaN do banco de dados
df_selecionado = df_selecionado.dropna()


In [105]:
# transformar variáveis nominais em categóricas
df_aux = df_selecionado
categoricas_var = ['sigla_uf_notificacao','sexo_paciente','raca_cor_paciente','escolaridade_paciente','gestante_paciente','possui_doenca_autoimune', 'possui_diabetes', 'possui_doencas_hematologicas', 
                     'possui_hepatopatias', 'possui_doenca_renal', 'possui_hipertensao', 
                     'possui_doenca_acido_peptica', 'apresenta_febre','apresenta_cefaleia','apresenta_exantema','apresenta_dor_costas','apresenta_mialgia','apresenta_vomito','apresenta_conjutivite','apresenta_dor_retroorbital',
                     'apresenta_artralgia','apresenta_artrite','apresenta_leucopenia','apresenta_petequias','classificacao_final','evolucao_caso','houve_obito']

for col in categoricas_var:
    df_selecionado[col]=pd.Categorical(df_selecionado[col])

df_selecionado

C:\Users\alexa\AppData\Local\Temp\ipykernel_10588\3418255390.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selecionado[col]=pd.Categorical(df_selecionado[col])
C:\Users\alexa\AppData\Local\Temp\ipykernel_10588\3418255390.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selecionado[col]=pd.Categorical(df_selecionado[col])
C:\Users\alexa\AppData\Local\Temp\ipykernel_10588\3418255390.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

,ano,sigla_uf_notificacao,idade_paciente,sexo_paciente,raca_cor_paciente,escolaridade_paciente,gestante_paciente,possui_doenca_autoimune,possui_diabetes,possui_doencas_hematologicas,...,apresenta_artralgia,apresenta_artrite,apresenta_leucopenia,apresenta_petequias,classificacao_final,evolucao_caso,houve_obito,intervalo_busca_atendimento,intervalo_sintoma_internacao,intervalo_internacao
118793,2024,AM,54,M,Parda,Educação superior completa,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,4,1,3
118794,2024,AM,48,F,Parda,Ensino médio completo,Não,Não,Não,Não,...,Não,Não,Não,Sim,Dengue Grave,Cura,não,4,1,22
118795,2024,AM,83,F,Parda,Ensino médio incompleto,Não,Não,Não,Não,...,Não,Não,Não,Não,Dengue com Sinais de Alarme,Cura,não,1,0,9
118808,2024,AC,42,F,Parda,Ensino médio completo,Não,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,2,0,19
118810,2024,AC,52,F,Parda,Ignorado,Não se aplica,Não,Não,Não,...,Sim,Sim,Sim,Sim,Dengue,Cura,não,4,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257950,2023,SP,45,F,Branca,Educação superior completa,Não,Não,Não,Não,...,Não,Não,Não,Não,Dengue Grave,Óbito pelo agravo,sim,1,6,3
257951,2023,SP,54,M,Branca,Ensino médio completo,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,1,0,9
257953,2023,SP,13,F,Branca,Ensino médio incompleto,Não se aplica,Não,Não,Não,...,Não,Não,Sim,Não,Dengue com Sinais de Alarme,Cura,não,3,4,25
257954,2023,SP,75,M,Branca,Ignorado,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,31,30,2


### Teste intervalo_internacao BeamSearch

In [106]:
# Carregar banco df_selecionado

data = df_selecionado


#target = ps.BinaryTarget ('sobrevivencia', True)
target2 = ps.NumericTarget('intervalo_internacao')
searchspace = ps.create_selectors(data, ignore=['intervalo_internacao'])
task = ps.SubgroupDiscoveryTask (
    data,
    target2,
    searchspace,
    result_set_size=60,
    depth=3,
    qf=ps.StandardQFNumeric(1.0))
result = ps.BeamSearch(beam_width=65).execute(task)
print(result.to_dataframe())

         quality                                           subgroup  size_sg  \
0   19866.437119  ano==2023 AND classificacao_final=='Dengue' AN...    20114   
1   19699.360518  classificacao_final=='Dengue' AND evolucao_cas...    20428   
2   19699.360518  classificacao_final=='Dengue' AND evolucao_cas...    20428   
3   19589.267149  classificacao_final=='Dengue' AND evolucao_cas...    20158   
4   19543.093750                         sigla_uf_notificacao=='MG'     5412   
5   19533.219703  classificacao_final=='Dengue' AND evolucao_cas...    20235   
6   19529.509146           ano==2023 AND sigla_uf_notificacao=='MG'     5328   
7   19421.248095  classificacao_final=='Dengue' AND evolucao_cas...    20210   
8   19401.030678  classificacao_final=='Dengue' AND evolucao_cas...    20331   
9   19270.084413  possui_doenca_acido_peptica=='Não' AND sigla_u...     5385   
10  19258.537538  ano==2023 AND possui_doenca_acido_peptica=='Nã...     5303   
11  19247.178925  possui_hepatopatias=='

In [107]:
result_beam_search = result.to_dataframe()
#Calculando o WRAcc do Beam Search
result_beam_search['WRAcc'] = (result_beam_search['size_sg']/result_beam_search['size_dataset'])*(result_beam_search['mean_sg']-result_beam_search['mean_dataset'])

### Teste intervalo_internacao DFS

In [108]:
# Carregar banco df_selecionado

data = df_selecionado


#target = ps.BinaryTarget ('sobrevivencia', True)
target2 = ps.NumericTarget('intervalo_internacao')
searchspace = ps.create_selectors(data, ignore=['intervalo_internacao'])
task = ps.SubgroupDiscoveryTask (
    data,
    target2,
    searchspace,
    result_set_size=60,
    depth=2,
    qf=ps.StandardQFNumeric(1.0))
result = ps.DFS().execute(task)
print(result.to_dataframe())

         quality                                           subgroup  size_sg  \
0   19699.360518  classificacao_final=='Dengue' AND evolucao_cas...    20428   
1   19543.093750                         sigla_uf_notificacao=='MG'     5412   
2   19529.509146           ano==2023 AND sigla_uf_notificacao=='MG'     5328   
3   19270.084413  possui_doenca_acido_peptica=='Não' AND sigla_u...     5385   
4   19247.178925  possui_hepatopatias=='Não' AND sigla_uf_notifi...     5337   
5   19121.358232  possui_doenca_autoimune=='Não' AND sigla_uf_no...     5320   
6   19027.697790  possui_doencas_hematologicas=='Não' AND sigla_...     5338   
7   18766.688453  possui_doenca_renal=='Não' AND sigla_uf_notifi...     5311   
8   18603.225991        ano==2023 AND classificacao_final=='Dengue'    21110   
9   18242.488948                      classificacao_final=='Dengue'    21442   
10  18156.641387  apresenta_conjutivite=='Não' AND sigla_uf_noti...     5282   
11  18078.791921  classificacao_final=='

In [109]:
result_dfs=result.to_dataframe()
#Calculando o WRAcc do DFS
result_dfs['WRAcc'] = (result_dfs['size_sg']/result_dfs['size_dataset'])*(result_dfs['mean_sg']-result_dfs['mean_dataset'])

### 	Teste intervalo_internação SSDC

In [110]:
df_selecionado.dtypes

ano                                int64
sigla_uf_notificacao            category
idade_paciente                     int32
sexo_paciente                   category
raca_cor_paciente               category
escolaridade_paciente           category
gestante_paciente               category
possui_doenca_autoimune         category
possui_diabetes                 category
possui_doencas_hematologicas    category
possui_hepatopatias             category
possui_doenca_renal             category
possui_hipertensao              category
possui_doenca_acido_peptica     category
apresenta_febre                 category
apresenta_cefaleia              category
apresenta_exantema              category
apresenta_dor_costas            category
apresenta_mialgia               category
apresenta_vomito                category
apresenta_conjutivite           category
apresenta_dor_retroorbital      category
apresenta_artralgia             category
apresenta_artrite               category
apresenta_leucop

In [111]:
task_name = "discovery"
target_type = "numeric"

# load class and fit to data
model = SSDC(task = task_name,max_depth=3, beam_width=25,max_rules=20,n_cutpoints=3)
model.fit(df_selecionado)

d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2  2  2 ... 24 24 24]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 0 ... 0 2 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14


KeyboardInterrupt: 

In [ ]:
print(model)
pd.DataFrame(model.antecedent_description)

IF x in sigla_uf_notificacao = SE AND raca_cor_paciente = Ignorado AND apresenta_artrite = Não THEN mean = 48.88805970149254; std = 19.5889438427202 ,  
ELSE IF x in sigla_uf_notificacao = GO AND gestante_paciente = Ignorado AND intervalo_busca_atendimento <= 3 THEN mean = 7.968; std = 4.857522002008843 ,  
ELSE IF x in sigla_uf_notificacao = SC AND apresenta_artrite = Sim AND intervalo_busca_atendimento <= 1 THEN mean = 8.49090909090909; std = 3.6282637060529597 ,  
ELSE IF x in sigla_uf_notificacao = MG AND houve_obito = sim AND 2 <= intervalo_sintoma_internacao <= 5 THEN mean = 39.4375; std = 24.98882562766806 ,  
ELSE IF x in sigla_uf_notificacao = SC AND possui_doencas_hematologicas = Sim THEN mean = 7.3076923076923075; std = 3.4478361278484573 ,  
ELSE IF x in evolucao_caso = Óbito por outras causas AND apresenta_vomito = Não AND apresenta_cefaleia = Sim THEN mean = 39.361111111111114; std = 26.701814221435395 ,  
ELSE IF x in sigla_uf_notificacao = SE AND possui_diabetes = Não A

,0
0,sigla_uf_notificacao = SE AND raca_cor_pacient...
1,sigla_uf_notificacao = GO AND gestante_pacient...
2,sigla_uf_notificacao = SC AND apresenta_artrit...
3,sigla_uf_notificacao = MG AND houve_obito = si...
4,sigla_uf_notificacao = SC AND possui_doencas_h...
5,evolucao_caso = Óbito por outras causas AND ap...
6,sigla_uf_notificacao = SE AND possui_diabetes ...
7,sigla_uf_notificacao = GO AND escolaridade_pac...
8,classificacao_final = Dengue com Sinais de Ala...
9,sigla_uf_notificacao = PE AND apresenta_leucop...


In [ ]:
model2 = SSDC(task = task_name, max_depth=4, beam_width=25,max_rules=20,n_cutpoints=3)
model2.fit(df_selecionado)

d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2  2  2 ... 24 24 24]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 0 ... 0 2 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60


In [ ]:
model3 = SSDC(task = task_name,max_depth=3, beam_width=25,max_rules=20,n_cutpoints=5)
model3.fit(df_selecionado)

d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2  2  2 ... 24 24 24]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 0 ... 0 2 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50


In [ ]:
model4 = SSDC(task = task_name,max_depth=4, beam_width=25,max_rules=20,n_cutpoints=5)
model4.fit(df_selecionado)

d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2  2  2 ... 24 24 24]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 0 ... 0 2 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63


In [ ]:
model5 = SSDC(task = task_name,max_depth=3, beam_width=50,max_rules=20,n_cutpoints=3)
model5.fit(df_selecionado)

d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2  2  2 ... 24 24 24]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 0 ... 0 2 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48


In [ ]:
model6 = SSDC(task = task_name,max_depth=4, beam_width=50,max_rules=20,n_cutpoints=3)
model6.fit(df_selecionado)


d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2  2  2 ... 24 24 24]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 0 ... 0 2 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\projeto_aprendizado_descritivo\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65


### Estatísticas de Qualidade por modelo no SSDC

#### Modelo 1

In [ ]:
model_estat = pd.DataFrame(model.statistic_rules)

In [ ]:
df_model = pd.DataFrame(model.antecedent_description, columns=['Subgrupros'])

In [ ]:
result_model = pd.concat([df_model, model_estat], axis=1)

In [ ]:
result_model['WRAcc'] = (result_model['usage']/df_selecionado.shape[0])*((result_model['mean'])-df_selecionado['intervalo_internacao'].mean())

In [ ]:
result_model

,Subgrupros,usage,mean,mean2,variance2,RSS2,variance,RSS_default_pattern,usage_default,RSS_default_uncovered,WRAcc
0,sigla_uf_notificacao = SE AND raca_cor_pacient...,134,48.888060,16.0,1.00,6.387526,383.726721,177440.537015,31354,9.342791e+06,0.134226
1,sigla_uf_notificacao = GO AND gestante_pacient...,125,7.968000,17.0,4.00,8.462983,23.595520,14984.907097,31229,9.325878e+06,-0.038078
2,sigla_uf_notificacao = SC AND apresenta_artrit...,55,8.490909,16.0,1.00,6.387526,13.164298,4881.678681,31174,9.320821e+06,-0.015861
3,sigla_uf_notificacao = MG AND houve_obito = si...,96,39.437500,16.0,9.00,22.387526,624.441406,104381.906114,31078,9.215500e+06,0.067823
4,sigla_uf_notificacao = SC AND possui_doencas_h...,26,7.307692,14.0,1.00,26.236611,11.887574,2918.662073,31052,9.212545e+06,-0.008518
5,evolucao_caso = Óbito por outras causas AND ap...,36,39.361111,16.0,4.00,12.387526,712.986883,44039.686210,31016,9.168090e+06,0.025396
6,sigla_uf_notificacao = SE AND possui_diabetes ...,78,37.974359,15.5,0.25,8.349797,534.522025,73466.874937,30938,9.093131e+06,0.051667
7,sigla_uf_notificacao = GO AND escolaridade_pac...,704,8.534091,17.5,0.25,0.500712,75.868705,110392.786732,30234,8.982117e+06,-0.208332
8,classificacao_final = Dengue com Sinais de Ala...,696,9.886494,17.5,0.25,0.500712,95.590503,106811.983961,29538,8.874542e+06,-0.178952
9,sigla_uf_notificacao = PE AND apresenta_leucop...,67,35.044776,17.0,1.00,2.462983,534.700156,57652.368927,29471,8.816315e+06,0.039930


#### Modelo 2

In [ ]:
model_estat2 = pd.DataFrame(model2.statistic_rules)
df_model2 = pd.DataFrame(model2.antecedent_description, columns=['Subgrupros'])
result_model2 = pd.concat([df_model2, model_estat2], axis=1)
result_model2['WRAcc'] = (result_model2['usage']/df_selecionado.shape[0])*((result_model2['mean'])-df_selecionado['intervalo_internacao'].mean())

#### Modelo 3

In [ ]:
model_estat3 = pd.DataFrame(model3.statistic_rules)
df_model3 = pd.DataFrame(model3.antecedent_description, columns=['Subgrupros'])
result_model3 = pd.concat([df_model3, model_estat3], axis=1)
result_model3['WRAcc'] = (result_model3['usage']/df_selecionado.shape[0])*((result_model3['mean'])-df_selecionado['intervalo_internacao'].mean())

#### Modelo 4

In [ ]:
model_estat4 = pd.DataFrame(model4.statistic_rules)
df_model4 = pd.DataFrame(model4.antecedent_description, columns=['Subgrupros'])
result_model4 = pd.concat([df_model4, model_estat4], axis=1)
result_model4['WRAcc'] = (result_model4['usage']/df_selecionado.shape[0])*((result_model4['mean'])-df_selecionado['intervalo_internacao'].mean())

#### Modelo 5

In [ ]:
model_estat5 = pd.DataFrame(model5.statistic_rules)
df_model5 = pd.DataFrame(model5.antecedent_description, columns=['Subgrupros'])
result_model5 = pd.concat([df_model5, model_estat5], axis=1)
result_model5['WRAcc'] = (result_model5['usage']/df_selecionado.shape[0])*((result_model5['mean'])-df_selecionado['intervalo_internacao'].mean())

#### Modelo 6

In [ ]:
model_estat6 = pd.DataFrame(model6.statistic_rules)
df_model6 = pd.DataFrame(model6.antecedent_description, columns=['Subgrupros'])
result_model6 = pd.concat([df_model6, model_estat6], axis=1)
result_model6['WRAcc'] = (result_model6['usage']/df_selecionado.shape[0])*((result_model6['mean'])-df_selecionado['intervalo_internacao'].mean())

### Salvando os experimentos

In [ ]:
experimentos = 'experimentos_sd.xlsx'

with pd.ExcelWriter(experimentos) as writer:
    result_model.to_excel(writer, sheet_name='SSD_modelo_1', index=False)
    result_model2.to_excel(writer, sheet_name='SSD_modelo_2', index=False)
    result_model3.to_excel(writer, sheet_name='SSD_modelo_3', index=False)
    result_model4.to_excel(writer, sheet_name='SSD_modelo_4', index=False)
    result_model5.to_excel(writer, sheet_name='SSD_modelo_5', index=False)
    result_model6.to_excel(writer, sheet_name='SSD_modelo_6', index=False)
    result_beam_search.to_excel(writer, sheet_name='Beam_Search', index=False)
    result_dfs.to_excel(writer, sheet_name='DFS', index=False)